### Frame the problem:
"describe the dataset, the purpose and aim of the project"

### Data loading, descriptive stats, percentage split of each column so we can verify after splitting
"add comments for every aspect here"

### Preprocessing - train/test split
"It may sound strange to voluntarily set aside part of the data at this stage. After all, you have only taken a quick glance at the data, and surely you should learn a whole lot more about it before you decide what algorithms to use, right? This is true, but your brain is an amazing pattern detection system, which means that it is highly prone to overfitting: if you look at the test set, you may stumble upon some seemingly interesting pattern in the test data that leads you to select a particular kind of Machine Learning model. When you estimate the generalization error using the test set, your estimate will be too optimistic and you will launch a system that will not perform as well as expected. This is called data snooping bias."

### Exploration and Descpritive study of training set (inc. viz)

correlations, scatterplots, histograms, all of Tim's stuff, some of George's stuff

### Data Cleaning
proving there is nothing to clean - just for the sake of including and evidencing.

### Dealing with categorical data

### Feature engineering - try and do different things here (??Maybe do this after baseline models??)

### Feature Scaling


(defining a function to combine scaling, engineering, and categorcial dealing and to apply to the test set)

### Naive Model
Predict the majority class for each observation (George)

### Baseline Model
Logistic Regression (Tim) 

### Classifiers: SVM, DT, RF, kNN, Gradient Boosting Classifier

SVM: Tim\
RF: Siyuan\
DT: Agam\
kNN: Tiana\
Gradient Boosting Classifier: George

##### Cross-Validation: Everyone does it for their own models

##### Recording Final Accuracies of Classifiers: 

### Deep Learning

##### Basic FF NN (Tim)

##### Hyperparameter and layer size tuning - Keras Tuner

##### ??? Bias and Variance checking - Use different tools such as batch normalisation ???

##### Accuracy comparison of NNs - choosing final NN

### Accuracies: Classifers vs NN 